In [ ]:
#import the necessary Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
from imblearn.over_sampling import SMOTE
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,f1_score,roc_curve
from sklearn.model_selection import KFold # import KFold
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/RTA Dataset.csv (1).csv") # Remove the trailing forward slash '/'

In [ ]:
for c in df.columns:
    print(c, df[c].unique()[:5])

In [ ]:
# convert the 'Date' column to datetime format
df['Time']= pd.to_datetime(df['Time'])

In [ ]:
df.duplicated()

In [ ]:
df.groupby('Accident_severity').size()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
df['Age_band_of_driver'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Age Band of Driver')
plt.xlabel('Age Band')
plt.ylabel('Count')

# Age_band_of_casualty
plt.subplot(1, 2, 2)
df['Age_band_of_casualty'].value_counts().sort_index().plot(kind='bar', color='salmon')
plt.title('Age Band of Casualty')
plt.xlabel('Age Band')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
df.hist(layout=(1,6), figsize=(30,5))
plt.show()


In [ ]:
df['Number_of_casualties'].value_counts()


In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(data=df, y='Number_of_vehicles_involved', x='Number_of_casualties')
plt.show()

In [ ]:
sns.boxplot(data=df, y='Number_of_casualties')
plt.show()


In [ ]:
sns.boxplot(data=df, y='Number_of_vehicles_involved')
plt.show()


In [ ]:
df['Number_of_vehicles_involved']

In [ ]:
sns.scatterplot(x=df['Number_of_vehicles_involved'], y=df['Number_of_casualties'])
plt.show()

In [ ]:
sns.pairplot(df[['Number_of_vehicles_involved','Number_of_casualties']])
plt.show()

In [ ]:
correlation_matrix = df[['Number_of_vehicles_involved','Number_of_casualties']].corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.pie(x=df['Accident_severity'].value_counts().values,
        labels=df['Accident_severity'].value_counts().index,
        autopct='%2.2f%%')
plt.show()

In [ ]:
#creating a facet grid with columns as survived=0 and survived=1
grid = sns.FacetGrid(data=df, col='Accident_severity', height=4, aspect=1, sharey=False)
# mapping bar plot and the data on to the grid
grid.map(sns.countplot, 'Number_of_vehicles_involved', palette=['black', 'brown', 'orange'])
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample DataFrame creation for demonstration
data = {'Time': ['2023-10-01 08:30:00', '2023-10-01 09:15:00', '2023-10-01 12:45:00']}
df = pd.DataFrame(data)

# Ensure the 'Time' column is in datetime format
df['Time'] = pd.to_datetime(df['Time'])

# Calculate TimeInSeconds
df['TimeInSeconds'] = df['Time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(df['TimeInSeconds'], bins=24, color='blue', edgecolor='black')
plt.title('Traffic Accident Time Distribution')
plt.xlabel('Time (Hours)')
plt.ylabel('Number of Accidents')
plt.xticks(range(0, 24 * 3600, 3600), range(24))  # Correctly labeling x-ticks in hours
plt.show()


In [ ]:
# dropping columns that can cause imbalance while imputation
lists = ['Vehicle_driver_relation', 'Work_of_casuality', 'Fitness_of_casuality',
         'Day_of_week', 'Casualty_severity', 'Time', 'Sex_of_driver',
         'Educational_level', 'Defect_of_vehicle', 'Owner_of_vehicle',
         'Service_year_of_vehicle', 'Road_surface_type', 'Sex_of_casualty']

df.drop(columns=lists, inplace=True, errors='ignore')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Fill missing values with mode column values, ensuring mode is valid
# Check if the column exists before attempting to fill missing values
for column in ['Driving_experience', 'Age_band_of_driver', 'Type_of_vehicle',
               'Area_accident_occured', 'Road_allignment', 'Type_of_collision',
               'Vehicle_movement', 'Lanes_or_Medians', 'Types_of_Junction']:
    if column in df.columns:  # Check if column is in the DataFrame
        mode_value = df[column].mode()
        if not mode_value.empty:
            df[column].fillna(mode_value[0], inplace=True)
        else:
            print(f"No mode found for column: {column}. Missing values remain.")
    else:
        print(f"Column '{column}' not found in DataFrame.")

In [ ]:
df.isnull().sum()

In [ ]:
df = df.rename(columns={"ExistingColumnName": "Accident_severity"})

In [ ]:
# Verify the existing column names
print(df.columns)

# Replace 'ExistingColumnName' with the actual name of the column you want to rename
df = df.rename(columns={"Time": "Accident_severity"})  # Example: Assuming "Time" is the column to rename

# Now you can access the renamed column
target_count = df['Accident_severity'].value_counts()
print('Class 0:', target_count[0])
print('Class 1:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)');

In [ ]:
#Encoding here
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df=df.apply(LE.fit_transform)

In [ ]:
 plt.figure(figsize=[25,15])
sns.heatmap(df.corr(),annot=True)


In [ ]:
for col in df.select_dtypes(include='object'):
    if df[col].nunique() <= 22:
        sns.countplot(y=col, data=df)
        plt.show()

In [ ]:
x = df.drop('Accident_severity', axis=1)
y = df['Accident_severity']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=42)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

In [ ]:
counter = Counter(ytrain)

print("=============================")

for k,v in counter.items():
    per = 100*v/len(ytrain)
    print(f"Class= {k}, n={v} ({per:.2f}%)")

oversample = SMOTE()
xtrain, ytrain = oversample.fit_resample(xtrain, ytrain)

counter = Counter(ytrain)

print("=============================")

for k,v in counter.items():
    per = 100*v/len(ytrain)
    print(f"Class= {k}, n={v} ({per:.2f}%)")

print("=============================")

print("Upsampled data shape: ", xtrain.shape, ytrain.shape)

In [ ]:
x=df.drop(columns=["Accident_severity"])
y=df["Accident_severity"]

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
models = {
    "LogisticRegression": OneVsRestClassifier(LogisticRegression(C=1.0, solver='liblinear', penalty='l2')),
    "DecisionTreeClassifier": OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5)),
    "KNeighborsClassifier": OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5, weights='distance', p=2)),
    "RandomForestClassifier": OneVsRestClassifier(RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=8)),
    "AdaBoostClassifier": OneVsRestClassifier(AdaBoostClassifier(n_estimators=200,learning_rate=0.1)),
    "GradientBoostingClassifier": OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5))
}

def plt_auc(ytest,ypred_proba):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(ytest == i, ypred_proba[:, i])

        roc_auc[i] = roc_auc_score(ytest == i, ypred_proba[:, i])


    plt.figure(figsize=(10, 6))
    for i, color in zip(range(3), ['aqua', 'darkorange', 'cornflowerblue']):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'ROC curve of class {i} (area = {roc_auc[i]:0.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
models = {
    # ... (other models)
    "KNeighborsClassifier": OneVsRestClassifier(KNeighborsClassifier(n_neighbors=2, weights='distance', p=2)), # or n_neighbors=3
    # ... (other models)
}

In [ ]:
models = {
    "LogisticRegression": OneVsRestClassifier(LogisticRegression(C=1.0, solver='liblinear', penalty='l2')),
    "DecisionTreeClassifier": OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy', max_depth=5)),
    "KNeighborsClassifier": OneVsRestClassifier(KNeighborsClassifier(n_neighbors=2, weights='distance')),
    "RandomForestClassifier": OneVsRestClassifier(RandomForestClassifier(n_estimators=100)),
    "AdaBoostClassifier": OneVsRestClassifier(AdaBoostClassifier(n_estimators=200)),
    "GradientBoostingClassifier": OneVsRestClassifier(GradientBoostingClassifier(n_estimators=100)),
}


In [ ]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)


In [ ]:
smote = SMOTE(sampling_strategy='minority', random_state=42, k_neighbors=1)
x, y = smote.fit_resample(x, y)


In [ ]:
!pip install -U imbalanced-learn scikit-learn


In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler

# Apply RandomOverSampler for rare class
ros = RandomOverSampler(random_state=42)
x, y = ros.fit_resample(x, y)

# Then use SMOTE for balancing
smote = SMOTE(random_state=42, k_neighbors=5)
x, y = smote.fit_resample(x, y)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Check class distribution
print(y.value_counts())

# If a class has only one sample, handle it using SMOTE:
smote = SMOTE(random_state=42, k_neighbors=1) # Setting k_neighbors to 1 for small samples
x, y = smote.fit_resample(x, y)

# Adjust test_size to be at least the number of classes
test_size = max(0.2, 3 / len(y))  # Ensure test_size is at least 3/len(y)

# Now perform train_test_split with the adjusted test_size
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42, stratify=y)